In [30]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Common Steps:
1. Remove Special Characters
2. Tokenize
3. Lemmatize/Stem
4. Remove Stop Words

In [6]:
subreddits = pd.read_csv('../data/subreddits_clean.csv')

In [8]:
subreddits.head(3)

,Unnamed: 0,title,selftext,subreddit,author,num_comments,score,timestamp,total_text,post_length_char,post_length_words
0,0,Answers to why,NaN,LifeProTips,AlienAgency,2,1,2020-07-17,Answers to why,15,3
1,1,¿Quieres obtener juegos y premios gratis en tu...,NaN,LifeProTips,GarbageMiserable0x0,2,1,2020-07-17,¿Quieres obtener juegos y premios gratis en tu...,60,10
2,2,Soothe digestion with lemongrass,NaN,LifeProTips,thaigrrrrrrr,2,1,2020-07-17,Soothe digestion with lemongrass,33,4


### Contractions

In [25]:
contractions = {
    "aren't": 'are not',
    "can't": 'cannot',
    "couldn't": "could not",
    "didn't": "did not",
    "doesnt": "does not",
    "don't": "do not",
    "hadn't": "had not", 
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "I'd": "i would",
    "I'll": "i will",
    "I am": "i'm",
    "I've": "i have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "she'd":"she would",
    "she'll": "she will",
    "she's": "she is",
    "there's": "there is",
    "they'd": "they had",
    "they'll":"they will",
    "they're": "they are", 
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "we've": "we have", 
    "weren't": "were not",
    "what's": "what is",
    "where's": "where is",
    "who'd": "who would",
    "who'll": "who will",
    "who's": "who is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [34]:
# Function Written By Dipanjan Sarkar 
# https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72

def expand_contractions(text, contraction_mapping=contractions):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions(subreddits['total_text'][0])

' Answers to why'

In [16]:
post_tokens = []

for post in subreddits['total_text']:
    post_tokens.append(word_tokenize(post.lower()))

In [22]:
lemmatizer = WordNetLemmatizer()

post_lemma_tokens = []

for token in post_tokens:
    running_lemma_tokens = []
    for word in token:
        running_lemma_tokens.append(lemmatizer.lemmatize(word))
    post_lemma_tokens.append(running_lemma_tokens)

In [28]:
post_lemma_tokens[4]

['if',
 'you',
 "'re",
 'ever',
 'lonely',
 ',',
 'try',
 'writing',
 '.',
 'whether',
 'it',
 "'s",
 'in',
 'a',
 'journal',
 ',',
 'a',
 'letter',
 'to',
 'someone',
 'or',
 'a',
 'story',
 '-',
 'the',
 'act',
 'of',
 'putting',
 'word',
 'to',
 'paper',
 'will',
 'make',
 'you',
 'feel',
 'le',
 'alone',
 ':',
 ')']